# Change detection using M3C2 algorithm
With the help of the change detection algorithm M3C2 we can remove non-static objects from the point cloud. Resulting in a point cloud with only static objects, that we name obstacles. The provided example tile in the folder `../datasets/pointclouds/m3c2/` is generated from the M3C2 plugin inside CloudCompare. This notebook presents how the M3C2 algorithm can be run using the CloudComPy package.

In [ ]:
import os
# os.environ["_CCTRACE_"]="ON"  # Uncomment to show CloudCompare debug output

import cloudComPy as cc
import numpy as np
import pandas as pd
import pathlib
import re
import time
from tqdm.notebook import tqdm # Requires tqdm and ipywidgets
import multiprocessing

In [ ]:
cc.initCC()  # to do once before using plugins or dealing with numpy

In [ ]:
if cc.isPluginM3C2():
    import cloudComPy.M3C2
else:
    print('M3C2 plugin not found.')

In [ ]:
base_folder = '../../datasets/Stadsdelen/'
pc_data_folder = 'pointclouds/'

# Resume previous incomplete run
resume = True

# Which stadsdelen to include
stadsdelen = ['centrum', 'haven', 'nieuw_west', 'noord', 'oost', 'west', 'zuid', 'zuid_oost']

# The distance analysis is executed on a number of points of interest called *core points*. 
# This could be e.g. the entire reference point cloud, a downsampled version of it, an equistant grid etc. 
# In the m3c2_params configuration file, we choose the entire point cloud:
param_file = '../datasets/m3c2_params.txt'

In [ ]:
# Find the number of CPU cores available on the system
disable_ht = True

max_thread_count = multiprocessing.cpu_count() - 2

if disable_ht:
    max_thread_count = int(max_thread_count / 2)

contents = ""

with open(param_file, 'r') as file :
    for line in file:
        if line.startswith('MaxThreadCount'):
            contents += f'MaxThreadCount={max_thread_count}\n'
        else:
            contents += f'{line}'

with open(param_file, 'w') as file:
    file.write(contents)

print(f'Will use {max_thread_count} CPU cores.')

In [ ]:
# Create folders for m3c2 files.
for stdsdl in stadsdelen:
    new_path = pathlib.Path(base_folder) / stdsdl / pc_data_folder / 'm3c2'
    new_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Get a list of all tilecodes for which we have two runs.
def get_tilecodes_from_folder(las_folder, las_prefix=''):
    """Get a set of unique tilecodes for the LAS files in a given folder."""
    files = pathlib.Path(las_folder).glob(f'{las_prefix}*.laz')
    tilecodes = set([re.match(r'.*(\d{4}_\d{4}).*', file.name)[1] for file in files])
    return tilecodes

all_tiles = {'stadsdeel': [],
             'tilecode': []}

for stdsdl in stadsdelen:
    path = pathlib.Path(base_folder) / stdsdl / pc_data_folder
    tiles = (get_tilecodes_from_folder(path / 'obstacles_run1/')
             .intersection(get_tilecodes_from_folder(path / 'obstacles_run2/')))
    if resume:
        done_tiles = get_tilecodes_from_folder(path / 'm3c2/', las_prefix='m3c2')
        tiles = tiles - done_tiles
    
    tiles = list(tiles)
    tiles.sort()
    all_tiles['stadsdeel'].extend([stdsdl]*len(tiles))
    all_tiles['tilecode'].extend(tiles)

all_tiles = pd.DataFrame(all_tiles)

In [ ]:
all_tiles.groupby('stadsdeel').agg('count')

In [ ]:
tile_tqdm = tqdm(all_tiles.index.values, unit='tile', smoothing=0)

for idx in tile_tqdm:
    stdsdl = all_tiles.loc[idx, 'stadsdeel']
    tilecode = all_tiles.loc[idx, 'tilecode']
    tile_tqdm.set_postfix_str(f'{stdsdl}/{tilecode}')
    
    time.sleep(0.1)
    
    path = pathlib.Path(base_folder) / stdsdl / pc_data_folder
    path_obst1 = path / 'obstacles_run1' / f'obst_{tilecode}.laz'
    path_obst2 = path / 'obstacles_run2' / f'obst_{tilecode}.laz'
    path_m3c2 = path / 'm3c2' / f'm3c2_{tilecode}.laz'
    
    cloud1 = cc.loadPointCloud(path_obst1.as_posix(), cc.CC_SHIFT_MODE.NO_GLOBAL_SHIFT)
    cloud2 = cc.loadPointCloud(path_obst2.as_posix(), cc.CC_SHIFT_MODE.NO_GLOBAL_SHIFT)

    m3c2_cloud = cc.M3C2.computeM3C2([cloud1, cloud2], param_file)

    if m3c2_cloud is None:
        # print(f'No M3C2 distances found for tile {tilecode}. Next...')
        continue # TODO handle
    # if m3c2_cloud.getNumberOfScalarFields() < 3:
    #     raise RuntimeError
    # dic = m3c2_cloud.getScalarFieldDic()
    # # The calculated distances
    # sf = m3c2_cloud.getScalarField(dic['M3C2 distance'])
    # if sf is None:
    #     raise RuntimeError

    cc.SavePointCloud(m3c2_cloud, path_m3c2.as_posix())
    
    cc.deleteEntity(cloud1)
    cc.deleteEntity(cloud2)
    cc.deleteEntity(m3c2_cloud)